In [22]:
# Package installations to work on WIRE

! pip install gensim
! pip install nltk
! pip install sentence_transformers
! pip install spacy
! python3 -m spacy download en_core_web_sm
! pip install bertopic
! pip install langchain


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
# imports

import csv
import json
import os
import re
import time
import unicodedata
import sys
import string
import pickle
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
ps = nltk.porter.PorterStemmer()

from sentence_transformers import SentenceTransformer
import spacy

from itertools import combinations
from collections import Counter
from scipy import stats

from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance, LangChain, TextGeneration
from hdbscan import HDBSCAN
import umap

import openai
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from langchain import OpenAI, PromptTemplate, LLMChain, HuggingFacePipeline, FewShotPromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.docstore.document import Document
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import SpectralEmbedding
from contextlib import contextmanager
from copy import deepcopy
from random import *
from pprint import pprint
from tqdm import tqdm
nltk.download(["names", "stopwords", "state_union", "twitter_samples", "movie_reviews", "averaged_perceptron_tagger", "vader_lexicon", "punkt", "wordnet"])

C:\Users\Seth Benson\AppData\Roaming\Python\Python311\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Seth Benson\AppData\Roaming\Python\Python311\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Seth Benson\AppData\Roaming\Python\Python311\site-pac

True

## Prepare LLM Model
- Create the Prompt
- Create an LLM (more than one option)
    - using a local LLM via LangChain's interface to HuggingFace
    - using GPT-4 as a chat completion through Azure

In [21]:
# Create a generic prompt template

template = '''
The following sentence is from a cable news transcript. Please classify whether the statement is POSITIVE, NEUTRAL, or NEGATIVE towards "{subject}". Only return the stance.

sentence: {statement}
'''

template2 = '''
The following sentence is from a cable news transcript. Is "{subject}" a main subject in the sentence? Respond only YES or NO.

sentence: {statement}
'''

stance_prompt = PromptTemplate(
    input_variables=["subject","statement"],
    template=template
)

relevance_prompt = PromptTemplate(
    input_variables=["subject","statement"],
    template=template2
)

LangChain's interface to HuggingFace

In [53]:
# Specify the LLM to use, Langchain verison

llm = HuggingFacePipeline.from_model_id(model_id="declare-lab/flan-alpaca-gpt4-xl", task = 'text2text-generation', device=0,
                                      model_kwargs={"max_length":500, "no_repeat_ngram_size":2})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GPT-4 as a chat completion through Azure

In [3]:
with open("aci_openai_credentials.txt") as f:
    open_api_base, open_api_key = f.read().split("\n")

openai.api_type = "azure"
openai.api_base = open_api_base
openai.api_version = "2023-03-15-preview"
openai.api_key = open_api_key

In [5]:
response = openai.ChatCompletion.create(
  engine="ACI_GPT-4",
  messages = [{"role":"user","content":stance_prompt.format(subject = "Donald Trump", statement="Those fires - Trump Didn't do that, ANTIFA did!")}],
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

In [6]:
response.choices[0].message['content']

'NEUTRAL'

## Organizing Transcript-Topics

In [25]:
#set up llm df
llm_df = pd.DataFrame(columns = ['subject','sentence','output'])
def transcript_cleaning(month):
    #import spacy stuff
    sid = SentimentIntensityAnalyzer()
    nlp = spacy.load("en_core_web_sm")
    NER = spacy.load("en_core_web_sm")

    tr_tp_st = [[], [], [], []]

    #read fox information, this is where ****file names**** and directories should be edited
    gl = pd.read_csv("../Data/foxGuestList.csv", encoding="windows-1252")
    fox = open("../Data/fox_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Sentence'] = lines['Line'].apply(lambda x: [str(sent) for sent in nlp(x).sents])
            sentences = lines.explode('Sentence')
            sentences = sentences.dropna(subset = ['Sentence'])

            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            text1 = NER(raw_text)
            words = []
            for word in text1.ents:
                #excluding unrelevant entities
                if word.label_ not in ['TIME','MONEY','PERCENT','DATE','QUANTITY','ORDINAL','CARDINAL']:
                    words.append(word.text)
            words = Counter(words).most_common()
            print(words)
            words = words[0:5]
            for w in words:
                #try:
                    #create df of all sentences including word
                    w1 = w[0]
                    wdf = sentences[sentences['Sentence'].astype(str).str.contains(str(w1))]
                    wdf = wdf.reset_index()
                    if len(wdf) > 2: #need at least three sentences to be added as an entity
                        w_list.append(w1)
                        sentence_list = list(wdf['Sentence'])
                        if len(wdf) > 15:
                            sentence_list = sample(sentence_list,15)
                        else:
                            pass
                        shuffle(sentence_list)
                        sent_list = []
                        #taking average (llm determined) sentiment from up to ten sentences with each entity
                        #for i, r in wdf.iterrows():
                            #if i < 10 and len(r['Line']) > i:
                        i = 0
                        for s in sentence_list:
                                if i > 10:
                                    break
                                try:
                                    print(s)#r['Sentence'])
                                    print(w1)
                                    response = openai.ChatCompletion.create(
                                            engine="ACI_GPT-4",
                                            messages = [{"role":"user","content":relevance_prompt.format(subject = w1, statement= s)}],
                                            temperature=0.7,
                                            max_tokens=800,
                                            top_p=0.95,
                                            frequency_penalty=0,
                                            presence_penalty=0,
                                            stop=None)
                                    relevance = response.choices[0].message['content']
                                    print(relevance)
                                    time.sleep(2)
                                    if relevance == "YES":
                                        i += 1
                                        response = openai.ChatCompletion.create(
                                            engine="ACI_GPT-4",
                                            messages = [{"role":"user","content":stance_prompt.format(subject = w1, statement= s)}],
                                            temperature=0.7,
                                            max_tokens=800,
                                            top_p=0.95,
                                            frequency_penalty=0,
                                            presence_penalty=0,
                                            stop=None)
                                        stance = response.choices[0].message['content']
                                        #stance = llm(stance_prompt.format(subject = w1, statement=r['Line']))

                                        #mapping llm output to int and storing
                                        def stance_value(sentence):  
                                            if 'NEGATIVE' in sentence:  
                                                return -1  
                                            elif 'POSITIVE' in sentence:  
                                                return 1  
                                            elif 'NEUTRAL' in sentence:  
                                                return 0  
                                            else:  
                                                return "No sentiment found in the sentence"  
                                        stance1 = stance_value(stance)
                                        sent_list.append(stance1)
                                        print(stance)
                                        print(stance1)
                                        llm_df.loc[len(llm_df)] = [w1,s,stance]
                                        time.sleep(1)
                                except Exception as error:
                                    print("Error:", error)
                                    time.sleep(2)

                        word_sentiment_list.append(sent_list)
                #except Exception as error:
                #    print("Error:", error)

            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    try:
                        tr_tp_st[0].append(l)
                        tr_tp_st[1].append(w_list[w])
                        tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                        tr_tp_st[3].append(lines["Program"][0])
                    except:
                        print("SOMETHING HAPPENED, WORD ADDING FAILED")

    return tr_tp_st

In [26]:
# Save out processed and topic modeled data
months = ['October']#['January','February','March','April','May','June','July','August','September','October','November','December']
for m in months:
    tr_tp_st = transcript_cleaning(m)
    tdf = pd.DataFrame(zip(tr_tp_st[0], tr_tp_st[1], tr_tp_st[2], tr_tp_st[3]), 
                       columns=["Transcript", "Topic", "Sentiment", "Program"],)
    tdf.to_csv('processed_transcripts_llm_' + m + '2020.csv')

[('Shannon', 21), ('Trump', 17), ('Biden', 16), ('Joe Biden', 9), ('the New York Times', 9), ('Twitter', 7), ('Philadelphia', 7), ('Florida', 6), ('Miles Taylor', 6), ('New Orleans', 6), ('Texas', 5), ('CNN', 5), ('Tony Bobulinski', 5), ('Michigan', 5), ('Pennsylvania', 4), ('Democratic', 4), ('Washington', 4), ('White House', 4), ('Americans', 4), ('COVID-19', 4), ('Arizona', 4), ('Wisconsin', 4), ('Georgia', 4), ('Nomiki', 4), ('Hurricane Zeta', 3), ('U.S.', 3), ('Ohio', 3), ('DC', 3), ('the New York Post', 3), ('Dorsey', 3), ('American', 3), ('Republicans', 3), ('North Carolina', 3), ('Leslie', 3), ('Europe', 3), ('Tampa', 2), ('Mississippi', 2), ('DHS', 2), ('New York Times', 2), ('Tucker Carlson', 2), ('Kushner', 2), ('Taylor', 2), ('Senate', 2), ('Facebook', 2), ('Rick Leventhal', 2), ('Google', 2), ('Ted Cruz', 2), ('Holocaust', 2), ('Steve Hilton', 2), ('Steve', 2), ('America', 2), ('China', 2), ('Rich Edson', 2), ('END VIDEO CLIP', 2), ('Fox News', 2), ('Iowa', 2), ('Delaware'

In [1]:
llm_df.to_csv("sentence_labels_October.csv")

NameError: name 'llm_df' is not defined

## Alternative transcrip sorting (using nouns and lines)

In [42]:
def transcript_cleaning(month):
    #import spacy stuff
    sid = SentimentIntensityAnalyzer()
    #nlp = spacy.load("en_core_web_sm")
    NER = spacy.load("en_core_web_sm")

    tr_tp_st = [[], [], [], []]

    gl = pd.read_csv("foxGuestList.csv", encoding="windows-1252")
    fox = open("fox_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Line'] = lines['Line'].apply(lambda x: x.lower())
            #lines["VADER"] = lines["Line"].apply(
            #    lambda x: list(sid.polarity_scores(x).values())[3]
            #)

            #getting most used nouns
            """word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            tokens = nltk.word_tokenize(raw_text)
            words = Counter(tokens).most_common()
            words1 = []
            i = 0
            while len(words1) < 15 and i < len(words):
                w = words[i][0]
                #print(w)
                tag =  nltk.pos_tag([w])
                t = tag[0][1] 
                if t in ['NN','NNS','NNP','NNPS','FW','SYM'] and w not in ['video','clip','i','end','begin']: #or t == "FW":
                    words1.append(w)
                i += 1"""
            
            #getting most used known entities
            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            text1 = NER(raw_text)
            words = []
            for word in text1.ents:
                if word.label_ not in ['TIME','MONEY','PERCENT','DATE','QUANTITY','ORDINAL','CARDINAL']:
                    words.append(word.text)
            words = Counter(words).most_common()
            words = words[0:15]

           #print(len(words))
            for w in words:
                #try:
                    w1 = w[0]
                    #print(w1)
                    wdf = lines[lines['Line'].astype(str).str.contains(str(w1))]
                    wdf = wdf.reset_index()
                    print(len(wdf))
                    w_list.append(w1)
                    sent_list = []
                    for i, r in wdf.iterrows():
                        if i < 5:
                            #r = wdf[i]
                            print(r['Line'])
                            print(w1)
                            stance = llm(stance_prompt.format(subject = w1, statement=r['Line']))
                            d = {'AGAINST': -1, 'NEUTRAL': 0, 'FOR': 1}
                            stance1 = d[stance]
                            sent_list.append(stance1)
                            print(stance)
                            print(stance1)
                    word_sentiment_list.append(sent_list)
                #except:
                #    pass

                
            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])


    return tr_tp_st